In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from utils import unpickle_object
from stan_models import plot_simple_model_results
from stan_models import Bayesian_model
from stan_models import sMAPE
from stan_models import make_prediction_stan_sampling
from processing import M5Data

['/Users/ryznerf/Documents/0_MIT/Spring_2022/0_Projects/1_Bayesian_Modelling_project/BayesianSales/main_code', '/Users/ryznerf/Documents/0_MIT/Spring_2022/0_Projects/1_Bayesian_Modelling_project/BayesianSales', '/Users/ryznerf/Documents/0_MIT/Spring_2022/0_Projects/1_Bayesian_Modelling_project/BayesianSales/data_processing', '/Users/ryznerf/Documents/0_MIT/Spring_2022/0_Projects/1_Bayesian_Modelling_project/BayesianSales/data_processing/data', '/Users/ryznerf/Documents/0_MIT/Spring_2022/0_Projects/1_Bayesian_Modelling_project/BayesianSales/data_processing/processing_code', '/Users/ryznerf/Documents/0_MIT/Spring_2022/0_Projects/1_Bayesian_Modelling_project/BayesianSales/main_code', '/Users/ryznerf/anaconda3/envs/stan/lib/python310.zip', '/Users/ryznerf/anaconda3/envs/stan/lib/python3.10', '/Users/ryznerf/anaconda3/envs/stan/lib/python3.10/lib-dynload', '', '/Users/ryznerf/anaconda3/envs/stan/lib/python3.10/site-packages', '/Users/ryznerf/Documents/0_MIT/Spring_2022/0_Projects/1_Bayesian

In [3]:
train_date_split='2014-06-01'

In [4]:
# initialize the dataset that was used for training
queried_item = "item_id == 'FOODS_1_096'"
model = Bayesian_model(day_start=1, day_end=1940)
_ = model.query_model_data(query=queried_item)

Using query: item_id == 'FOODS_1_096'
Processed data for query: item_id == 'FOODS_1_096'


In [7]:
train = model.create_train_data(train_date_split)

In [8]:
train

,store_id,date,state_id,demand,sell_price,log_Q,log_P,P,Q,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10,m_11,m_12,q_1,q_2,q_3,q_4
0,CA_1,2011-01-30,CA,0.0,6.22,-69.077553,1.827770,6.22,0.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,CA_1,2011-02-06,CA,1.0,6.22,0.000000,1.827770,6.22,1.0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,CA_1,2011-02-13,CA,1.0,6.22,0.000000,1.827770,6.22,1.0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,CA_1,2011-02-20,CA,1.0,6.22,0.000000,1.827770,6.22,1.0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,CA_1,2011-02-27,CA,1.0,6.22,0.000000,1.827770,6.22,1.0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2672,WI_3,2014-05-04,WI,57.0,7.89,4.043051,2.065596,7.89,57.0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
2673,WI_3,2014-05-11,WI,61.0,7.89,4.110874,2.065596,7.89,61.0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
2674,WI_3,2014-05-18,WI,68.0,7.89,4.219508,2.065596,7.89,68.0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
2675,WI_3,2014-05-25,WI,29.0,7.91,3.367296,2.068128,7.91,29.0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0


In [18]:
cluster_level = 'store_id'
store_labels = train[cluster_level].unique()
ticker_dic = {}
for cluster, store in enumerate(store_labels):
    ticker_dic[store] = cluster

train['cluster'] = train[cluster_level].apply(lambda x: ticker_dic[x])

In [21]:
train

,store_id,date,state_id,demand,sell_price,log_Q,log_P,P,Q,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10,m_11,m_12,q_1,q_2,q_3,q_4,cluster
0,CA_1,2011-01-30,CA,0.0,6.22,-69.077553,1.827770,6.22,0.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,CA_1,2011-02-06,CA,1.0,6.22,0.000000,1.827770,6.22,1.0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,CA_1,2011-02-13,CA,1.0,6.22,0.000000,1.827770,6.22,1.0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,CA_1,2011-02-20,CA,1.0,6.22,0.000000,1.827770,6.22,1.0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,CA_1,2011-02-27,CA,1.0,6.22,0.000000,1.827770,6.22,1.0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2672,WI_3,2014-05-04,WI,57.0,7.89,4.043051,2.065596,7.89,57.0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,9
2673,WI_3,2014-05-11,WI,61.0,7.89,4.110874,2.065596,7.89,61.0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,9
2674,WI_3,2014-05-18,WI,68.0,7.89,4.219508,2.065596,7.89,68.0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,9
2675,WI_3,2014-05-25,WI,29.0,7.91,3.367296,2.068128,7.91,29.0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,9
